In [49]:
import pandas as pd
from datetime import datetime, date
from dateutil.relativedelta import relativedelta
from typing import List
import re

In [50]:
def generate_year_month_range(end_date: date = None, years_back: int = 6) -> List[str]:
    """
    Generate a list of year-month combinations in 'yyyymm' format,
    starting from the specified end date and going back a specified number of years.

    Args:
        end_date (date, optional): The end date to start from. Defaults to today's date.
        years_back (int, optional): Number of years to go back. Defaults to 6.

    Returns:
        List[str]: List of year-month combinations in 'yyyymm' format, sorted in descending order.

    Example:
        >>> generate_year_month_range()  # If today is 2024-11-14
        ['202411', '202410', '202409', ..., '201812']
    """
    # If no end date is provided, use today's date
    if end_date is None:
        end_date = date.today()

    # Calculate start date
    start_date = end_date - relativedelta(years=years_back)

    # Initialize result list
    date_list = []

    # Current date for iteration
    current_date = end_date

    # Generate dates until we reach start date
    while current_date >= start_date:
        # Format date as 'yyyymm'
        date_str = current_date.strftime('%Y%m')
        date_list.append(date_str)
        # Move to previous month
        current_date -= relativedelta(months=1)

    return date_list

# Example usage
if __name__ == "__main__":
    date_list = generate_year_month_range()
    print(f"Generated {len(date_list)} year-month combinations:")
    print(date_list[:12])  # Print first year as example

Generated 73 year-month combinations:
['202502', '202501', '202412', '202411', '202410', '202409', '202408', '202407', '202406', '202405', '202404', '202403']


In [51]:
import requests

In [52]:
# read in the metro_areas.json file as a list
import json
with open('data/metro_areas.json') as f:
    metro_areas = json.load(f)

metro_areas

['Abilene, TX',
 'Akron, OH',
 'Albany, OR',
 'Albany-Schenectady-Troy, NY',
 'Albuquerque, NM',
 'Allentown-Bethlehem-Easton, PA-NJ',
 'Altoona, PA',
 'Amarillo, TX',
 'Ames, IA',
 'Amherst Town-Northampton, MA',
 'Ann Arbor, MI',
 'Appleton, WI',
 'Asheville, NC',
 'Athens-Clarke County, GA',
 'Atlanta-Sandy Springs-Roswell, GA',
 'Atlantic City-Hammonton, NJ',
 'Auburn-Opelika, AL',
 'Baltimore-Columbia-Towson, MD',
 'Barnstable Town, MA',
 'Baton Rouge, LA',
 'Battle Creek, MI',
 'Bay City, MI',
 'Birmingham, AL',
 'Bloomington, IL',
 'Bloomington, IN',
 'Boise City, ID',
 'Boston-Cambridge-Newton, MA-NH',
 'Boulder, CO',
 'Bowling Green, KY',
 'Bremerton-Silverdale-Port Orchard, WA',
 'Bridgeport-Stamford-Danbury, CT',
 'Brownsville-Harlingen, TX',
 'Buffalo-Cheektowaga, NY',
 'Burlington, NC',
 'Burlington-South Burlington, VT',
 'Canton-Massillon, OH',
 'Cape Coral-Fort Myers, FL',
 'Cape Girardeau, MO-IL',
 'Carson City, NV',
 'Cedar Rapids, IA',
 'Chambersburg, PA',
 'Champaig

In [53]:
import pandas as pd
import requests
from datetime import datetime

def check_url_exists(url):
    """Check if a URL exists without downloading the full file"""
    try:
        response = requests.head(url)
        return response.status_code == 200
    except:
        return False

def process_2024_data(url):
    """Process data from 2024 onwards"""
    df = pd.read_excel(url, skiprows=7)
    
    # Clean columns
    df.drop(columns=['Metro /Micro Code', 'Unnamed: 10'], inplace=True)
    
    # Clean column names and data
    df.columns = df.columns.str.replace('.1', '_ytd')
    df['Name'] = df['Name'].str.rstrip()
    
    # Add date
    df['date'] = url[-10:-4]
    
    return df

def process_2022_2023_data(url):
    """Process data from 2022-2023"""
    df = pd.read_excel(url, skiprows=7)
    
    # Remove first row and clean columns
    df = df.iloc[1:]
    df.drop(columns=['Unnamed: 9'], inplace=True)
    
    # Clean column names and data
    df.columns = df.columns.str.replace('.1', '_ytd')
    df['Name'] = df['Name'].str.rstrip()
    
    # Add date
    df['date'] = url[-10:-4]
    
    return df

def process_pre_2022_data(url):
    """Process data from 2021 and earlier"""
    df = pd.read_excel(url, skiprows=7)
    
    # Remove first row and clean columns
    df = df.iloc[1:]
    df.drop(columns=['Monthly Coverage Percent*', 'Unnamed: 10'], inplace=True)
    
    # Clean column names and data
    df.columns = df.columns.str.replace('.1', '_ytd')
    df['Name'] = df['Name'].str.rstrip()
    
    # Add date
    df['date'] = url[-10:-4]
    
    return df

# Initialize an empty list to store all dataframes
all_dfs = []

# Process each date
for date_str in date_list:
    year = int(date_str[:4])
    
    try:
        # Determine URL format and processing function based on year
        if year >= 2024:
            url = f'https://www.census.gov/construction/bps/xls/cbsamonthly_{date_str}.xls'
            process_func = process_2024_data
        else:
            url = f'https://www.census.gov/construction/bps/xls/msamonthly_{date_str}.xls'
            if year >= 2022:
                process_func = process_2022_2023_data
            else:
                process_func = process_pre_2022_data
        
        # Check if URL exists
        if check_url_exists(url):
            try:
                df = process_func(url)
                all_dfs.append(df)
                print(f"Successfully processed data for {date_str}")
            except Exception as e:
                print(f"Error processing {date_str}: {str(e)}")
                continue
        else:
            print(f"No data available for {date_str}")
            continue
            
    except Exception as e:
        print(f"Error with {date_str}: {str(e)}")
        continue

# Concatenate all dataframes if we have any data
if all_dfs:
    homebuilding = pd.concat(all_dfs, ignore_index=True)
    
    # Convert date column to datetime
    homebuilding['date'] = pd.to_datetime(homebuilding['date'], format='%Y%m')
    
    # Sort by date and other relevant columns
    homebuilding = homebuilding.sort_values(['date', 'Name'], ascending=[False, True])
    
    print(f"\nFinal dataset contains {len(homebuilding)} rows from {len(all_dfs)} different months")
    print(f"Date range: {homebuilding['date'].min()} to {homebuilding['date'].max()}")
else:
    print("No data was successfully processed")
    homebuilding = pd.DataFrame()

No data available for 202502
No data available for 202501
Successfully processed data for 202412
Successfully processed data for 202411
Successfully processed data for 202410
Successfully processed data for 202409
Successfully processed data for 202408
Successfully processed data for 202407
Successfully processed data for 202406
Successfully processed data for 202405
Successfully processed data for 202404
Successfully processed data for 202403
Successfully processed data for 202402
Successfully processed data for 202401
Successfully processed data for 202312
Successfully processed data for 202311
Successfully processed data for 202310
Successfully processed data for 202309
Successfully processed data for 202308
Successfully processed data for 202307
Successfully processed data for 202306
Successfully processed data for 202305
Successfully processed data for 202304
Successfully processed data for 202303
Successfully processed data for 202302
Successfully processed data for 202301
Succes

In [54]:
# homebuilding['date'] = pd.to_datetime(homebuilding['date'])
homebuilding.dtypes

CSA                                                  float64
CBSA                                                 float64
Name                                                  object
Total                                                float64
1 Unit                                               float64
2 Units                                              float64
3 and 4 Units                                        float64
5 Units or More                                      float64
Num of Structures With 5 Units or More               float64
Total_ytd                                            float64
1 Unit_ytd                                           float64
2 Units_ytd                                          float64
3 and 4 Units_ytd                                    float64
5 Units or More_ytd                                  float64
Num of Structures With 5 Units or More_ytd           float64
date                                          datetime64[ns]
dtype: object

In [7]:
homebuilding.columns

Index(['CSA', 'CBSA', 'Name', 'Total', '1 Unit', '2 Units', '3 and 4 Units',
       '5 Units or More', 'Num of Structures With 5 Units or More',
       'Total_ytd', '1 Unit_ytd', '2 Units_ytd', '3 and 4 Units_ytd',
       '5 Units or More_ytd', 'Num of Structures With 5 Units or More_ytd',
       'date'],
      dtype='object')

In [55]:
# create a 'multi_total' column that sums the columns for different types of multi-unit structures
homebuilding['multi_total'] = homebuilding['2 Units'] + homebuilding['3 and 4 Units'] + homebuilding['5 Units or More'] + homebuilding['Num of Structures With 5 Units or More']

# create a 'multi_total_ytd' column that sums the columns for different types of multi-unit structures
homebuilding['multi_total_ytd'] =  homebuilding['2 Units_ytd'] + homebuilding['3 and 4 Units_ytd'] + homebuilding['5 Units or More_ytd'] + homebuilding['Num of Structures With 5 Units or More_ytd']

In [56]:
# rename Name to name, Total to total, and Total_ytd to total_ytd
homebuilding.rename(
    columns={
        'Name': 'name',
        'Total': 'total',
        'Total_ytd': 'total_ytd'
    },
    inplace=True
)

homebuilding = homebuilding[['date', 'name', 'total', 'total_ytd', 'multi_total', 'multi_total_ytd']]

homebuilding

,date,name,total,total_ytd,multi_total,multi_total_ytd
0,2024-12-01,"Aberdeen, SD",1.0,88.0,0.0,16.0
1,2024-12-01,"Aberdeen, WA",36.0,297.0,5.0,44.0
2,2024-12-01,"Abilene, TX",78.0,520.0,0.0,96.0
3,2024-12-01,"Ada, OK",0.0,2.0,0.0,0.0
4,2024-12-01,"Adrian, MI",9.0,137.0,0.0,0.0
...,...,...,...,...,...,...
29831,2019-11-01,"Worcester, MA-CT",18.0,219.0,2.0,52.0
29832,2019-11-01,"Yakima, WA",52.0,823.0,13.0,476.0
29833,2019-11-01,"York-Hanover, PA",11.0,155.0,4.0,62.0
29834,2019-11-01,"Youngstown-Warren-Boardman, OH-PA",13.0,200.0,0.0,7.0


In [57]:
# find nyc data
homebuilding[homebuilding['name'].str.contains('New York')]

,date,name,total,total_ytd,multi_total,multi_total_ytd
596,2024-12-01,"New York-Newark-Jersey City, NY-NJ",6341.0,58580.0,5599.0,46990.0
1517,2024-11-01,"New York-Newark-Jersey City, NY-NJ",4216.0,52213.0,3448.0,41343.0
2438,2024-10-01,"New York-Newark-Jersey City, NY-NJ",4168.0,48063.0,2924.0,37914.0
3359,2024-09-01,"New York-Newark-Jersey City, NY-NJ",3666.0,43928.0,2683.0,35127.0
4280,2024-08-01,"New York-Newark-Jersey City, NY-NJ",5023.0,40252.0,3890.0,32418.0
...,...,...,...,...,...,...
28235,2020-03-01,"New York-Newark-Jersey City, NY-NJ-PA",3410.0,12366.0,2461.0,9753.0
28603,2020-02-01,"New York-Newark-Jersey City, NY-NJ-PA",2929.0,8873.0,2147.0,7250.0
28971,2020-01-01,"New York-Newark-Jersey City, NY-NJ-PA",5982.0,5982.0,5169.0,5169.0
29339,2019-12-01,"New York-Newark-Jersey City, NY-NJ-PA",5708.0,60746.0,5096.0,50701.0


In [62]:
# clean up the name column
homebuilding['name'] = homebuilding['name'].apply(lambda x: x.split(',')[0].split('-')[0] + ',' + x.split(',')[1].split('-')[0])

homebuilding

,date,name,total,total_ytd,multi_total,multi_total_ytd,name_clean
0,2024-12-01,"Aberdeen, SD",1.0,88.0,0.0,16.0,"Aberdeen, SD"
1,2024-12-01,"Aberdeen, WA",36.0,297.0,5.0,44.0,"Aberdeen, WA"
2,2024-12-01,"Abilene, TX",78.0,520.0,0.0,96.0,"Abilene, TX"
3,2024-12-01,"Ada, OK",0.0,2.0,0.0,0.0,"Ada, OK"
4,2024-12-01,"Adrian, MI",9.0,137.0,0.0,0.0,"Adrian, MI"
...,...,...,...,...,...,...,...
29831,2019-11-01,"Worcester, MA",18.0,219.0,2.0,52.0,"Worcester, MA"
29832,2019-11-01,"Yakima, WA",52.0,823.0,13.0,476.0,"Yakima, WA"
29833,2019-11-01,"York, PA",11.0,155.0,4.0,62.0,"York, PA"
29834,2019-11-01,"Youngstown, OH",13.0,200.0,0.0,7.0,"Youngstown, OH"


In [63]:
# sort by date with earliest date first
homebuilding = homebuilding.sort_values('date')



homebuilding

,date,name,total,total_ytd,multi_total,multi_total_ytd,name_clean
29835,2019-11-01,"Yuma, AZ",80.0,1063.0,0.0,6.0,"Yuma, AZ"
29584,2019-11-01,"Fayetteville, AR",495.0,6303.0,199.0,2562.0,"Fayetteville, AR"
29585,2019-11-01,"Flagstaff, AZ",48.0,650.0,19.0,194.0,"Flagstaff, AZ"
29586,2019-11-01,"Flint, MI",18.0,371.0,0.0,130.0,"Flint, MI"
29587,2019-11-01,"Florence, SC",36.0,664.0,6.0,288.0,"Florence, SC"
...,...,...,...,...,...,...,...
608,2024-12-01,"Oak Harbor, WA",19.0,402.0,0.0,112.0,"Oak Harbor, WA"
607,2024-12-01,"Norwich, CT",11.0,487.0,0.0,259.0,"Norwich, CT"
606,2024-12-01,"Norwalk, OH",3.0,40.0,0.0,0.0,"Norwalk, OH"
604,2024-12-01,"North Port, FL",1075.0,14938.0,279.0,4128.0,"North Port, FL"


In [65]:
# Ensure the DataFrame is sorted by 'name' and 'date'
homebuilding = homebuilding.sort_values(by=['name', 'date'])

# Create a 12-month running total column
homebuilding['rt'] = (
    homebuilding.groupby('name')['total']
    .rolling(window=12, min_periods=1)
    .sum()
    .reset_index(level=0, drop=True)
)

# Create a 12-month running total column for multi-unit structures
homebuilding['multi_rt'] = (
    homebuilding.groupby('name')['multi_total']
    .rolling(window=12, min_periods=1)
    .sum()
    .reset_index(level=0, drop=True)
)

homebuilding

,date,name,total,total_ytd,multi_total,multi_total_ytd,rt,multi_rt
10131,2024-01-01,"Aberdeen, SD",2.0,2.0,0.0,0.0,2.0,0.0
9210,2024-02-01,"Aberdeen, SD",2.0,4.0,0.0,0.0,4.0,0.0
8289,2024-03-01,"Aberdeen, SD",4.0,8.0,0.0,0.0,8.0,0.0
7368,2024-04-01,"Aberdeen, SD",15.0,23.0,0.0,0.0,23.0,0.0
6447,2024-05-01,"Aberdeen, SD",1.0,24.0,0.0,0.0,24.0,0.0
...,...,...,...,...,...,...,...,...
4604,2024-08-01,"Zanesville, OH",17.0,82.0,14.0,65.0,84.0,65.0
3683,2024-09-01,"Zanesville, OH",15.0,97.0,12.0,77.0,99.0,77.0
2762,2024-10-01,"Zanesville, OH",16.0,113.0,14.0,91.0,115.0,91.0
1841,2024-11-01,"Zanesville, OH",15.0,128.0,14.0,105.0,130.0,105.0


In [66]:
# show the df sorted by multi_total_12_month_running in the max date
homebuilding[homebuilding['date'] == homebuilding['date'].max()].sort_values('multi_rt', ascending=False)

,date,name,total,total_ytd,multi_total,multi_total_ytd,rt,multi_rt
596,2024-12-01,"New York, NY",6341.0,58580.0,5599.0,46990.0,59629.0,48211.0
206,2024-12-01,"Dallas, TX",6144.0,72151.0,2604.0,25721.0,70339.0,24897.0
56,2024-12-01,"Austin, TX",3073.0,32304.0,2020.0,16138.0,33530.0,16766.0
489,2024-12-01,"Los Angeles, CA",2527.0,26671.0,1526.0,15374.0,27246.0,15598.0
653,2024-12-01,"Phoenix, AZ",2322.0,45515.0,676.0,15735.0,44491.0,15002.0
...,...,...,...,...,...,...,...,...
237,2024-12-01,"Duncan, OK",1.0,17.0,0.0,0.0,17.0,0.0
740,2024-12-01,"Sanford, NC",62.0,602.0,0.0,0.0,602.0,0.0
561,2024-12-01,"Morgantown, WV",0.0,25.0,0.0,16.0,13.0,0.0
77,2024-12-01,"Beeville, TX",1.0,15.0,0.0,0.0,14.0,0.0


In [68]:
# population

# read in the population data
metros = pd.read_csv('data/zori_metro_long_clean.csv')

# change 'date' to datetime
metros['date'] = pd.to_datetime(metros['date'])

metros

,name,date,zori,lat,lng,pop_2023
0,"New York, NY",2015-02-01,2225.296911,40.6943,-73.9249,19498249.0
1,"Los Angeles, CA",2015-02-01,1802.281380,34.1141,-118.4068,12799100.0
2,"Chicago, IL",2015-02-01,1352.236287,41.8375,-87.6866,9262825.0
3,"Dallas, TX",2015-02-01,1082.375956,32.7935,-96.7667,8100037.0
4,"Houston, TX",2015-02-01,1221.669822,29.7860,-95.3885,7510253.0
...,...,...,...,...,...,...
5995,"Salt Lake City, UT",2025-01-01,1656.973108,40.7776,-111.9311,1267864.0
5996,"Hartford, CT",2025-01-01,1870.252402,41.7661,-72.6834,1151543.0
5997,"Buffalo, NY",2025-01-01,1324.995704,42.9018,-78.8487,1155604.0
5998,"Birmingham, AL",2025-01-01,1339.420553,33.5279,-86.7971,1184290.0


In [69]:
# merge the population data with the homebuilding data
# merge on 'name' and 'date'
# only include rows where both 'name' and 'date' are in both dataframes
homebuilding_zori = pd.merge(
    homebuilding,
    metros,
    on=['name', 'date'],
    how='inner'
)

homebuilding_zori

,date,name,total,total_ytd,multi_total,multi_total_ytd,rt,multi_rt,zori,lat,lng,pop_2023
0,2019-11-01,"Atlanta, GA",1819.0,30513.0,117.0,6602.0,1819.0,117.0,1371.279930,33.7628,-84.4220,6307261.0
1,2019-12-01,"Atlanta, GA",2388.0,32729.0,213.0,6732.0,4207.0,330.0,1368.872179,33.7628,-84.4220,6307261.0
2,2020-01-01,"Atlanta, GA",2912.0,2912.0,973.0,973.0,7119.0,1303.0,1368.798323,33.7628,-84.4220,6307261.0
3,2020-02-01,"Atlanta, GA",2742.0,5661.0,651.0,1562.0,9861.0,1954.0,1373.768759,33.7628,-84.4220,6307261.0
4,2020-03-01,"Atlanta, GA",2216.0,7883.0,127.0,1633.0,12077.0,2081.0,1382.102627,33.7628,-84.4220,6307261.0
...,...,...,...,...,...,...,...,...,...,...,...,...
3095,2024-08-01,"Washington, DC",2137.0,15660.0,1257.0,7569.0,22460.0,11575.0,2375.472810,38.9047,-77.0163,6304975.0
3096,2024-09-01,"Washington, DC",900.0,16477.0,222.0,7747.0,21885.0,11089.0,2383.224478,38.9047,-77.0163,6304975.0
3097,2024-10-01,"Washington, DC",1801.0,19004.0,922.0,8651.0,21838.0,10897.0,2382.780218,38.9047,-77.0163,6304975.0
3098,2024-11-01,"Washington, DC",1124.0,20058.0,318.0,8903.0,21457.0,10369.0,2372.798426,38.9047,-77.0163,6304975.0


In [70]:
homebuilding_zori['name'].nunique()

50

In [71]:
homebuilding_zori.columns

Index(['date', 'name', 'total', 'total_ytd', 'multi_total', 'multi_total_ytd',
       'rt', 'multi_rt', 'zori', 'lat', 'lng', 'pop_2023'],
      dtype='object')

In [72]:
# create 'total_per_capita' and 'total_ytd_per_capita' columns that divide 'total' and 'total_ytd' by 'pop_2023'
homebuilding_zori['total_pc'] = homebuilding_zori['total'] / homebuilding_zori['pop_2023']
homebuilding_zori['total_ytd_pc'] = homebuilding_zori['total_ytd'] / homebuilding_zori['pop_2023']
homebuilding_zori['multi_total_pc'] = homebuilding_zori['multi_total'] / homebuilding_zori['pop_2023']
homebuilding_zori['multi_total_ytd_pc'] = homebuilding_zori['multi_total_ytd'] / homebuilding_zori['pop_2023']
homebuilding_zori['rt_pc'] = homebuilding_zori['rt'] / homebuilding_zori['pop_2023']
homebuilding_zori['multi_rt_pc'] = homebuilding_zori['multi_rt'] / homebuilding_zori['pop_2023']


# change per capita columns to per 1000
for col in homebuilding_zori.columns:
    if 'pc' in col:
        homebuilding_zori[col] = homebuilding_zori[col] * 1000

homebuilding_zori

,date,name,total,total_ytd,multi_total,multi_total_ytd,rt,multi_rt,zori,lat,lng,pop_2023,total_pc,total_ytd_pc,multi_total_pc,multi_total_ytd_pc,rt_pc,multi_rt_pc
0,2019-11-01,"Atlanta, GA",1819.0,30513.0,117.0,6602.0,1819.0,117.0,1371.279930,33.7628,-84.4220,6307261.0,0.288398,4.837758,0.018550,1.046730,0.288398,0.018550
1,2019-12-01,"Atlanta, GA",2388.0,32729.0,213.0,6732.0,4207.0,330.0,1368.872179,33.7628,-84.4220,6307261.0,0.378611,5.189099,0.033771,1.067341,0.667009,0.052321
2,2020-01-01,"Atlanta, GA",2912.0,2912.0,973.0,973.0,7119.0,1303.0,1368.798323,33.7628,-84.4220,6307261.0,0.461690,0.461690,0.154267,0.154267,1.128699,0.206587
3,2020-02-01,"Atlanta, GA",2742.0,5661.0,651.0,1562.0,9861.0,1954.0,1373.768759,33.7628,-84.4220,6307261.0,0.434737,0.897537,0.103214,0.247651,1.563436,0.309802
4,2020-03-01,"Atlanta, GA",2216.0,7883.0,127.0,1633.0,12077.0,2081.0,1382.102627,33.7628,-84.4220,6307261.0,0.351341,1.249829,0.020136,0.258908,1.914777,0.329937
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3095,2024-08-01,"Washington, DC",2137.0,15660.0,1257.0,7569.0,22460.0,11575.0,2375.472810,38.9047,-77.0163,6304975.0,0.338939,2.483753,0.199366,1.200481,3.562266,1.835852
3096,2024-09-01,"Washington, DC",900.0,16477.0,222.0,7747.0,21885.0,11089.0,2383.224478,38.9047,-77.0163,6304975.0,0.142744,2.613333,0.035210,1.228712,3.471068,1.758770
3097,2024-10-01,"Washington, DC",1801.0,19004.0,922.0,8651.0,21838.0,10897.0,2382.780218,38.9047,-77.0163,6304975.0,0.285647,3.014128,0.146234,1.372091,3.463614,1.728318
3098,2024-11-01,"Washington, DC",1124.0,20058.0,318.0,8903.0,21457.0,10369.0,2372.798426,38.9047,-77.0163,6304975.0,0.178272,3.181297,0.050436,1.412060,3.403186,1.644574


In [73]:
# create 'zori_yoy' column that calculates the year-over-year change in 'zori' for each metro area
homebuilding_zori['zori_yoy'] = homebuilding_zori.groupby('name')['zori'].pct_change(12)
homebuilding_zori

/var/folders/jd/5wy1jytx2pg8j4jr12tl0k_m0000gn/T/ipykernel_56554/3262750924.py:2: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  homebuilding_zori['zori_yoy'] = homebuilding_zori.groupby('name')['zori'].pct_change(12)


,date,name,total,total_ytd,multi_total,multi_total_ytd,rt,multi_rt,zori,lat,lng,pop_2023,total_pc,total_ytd_pc,multi_total_pc,multi_total_ytd_pc,rt_pc,multi_rt_pc,zori_yoy
0,2019-11-01,"Atlanta, GA",1819.0,30513.0,117.0,6602.0,1819.0,117.0,1371.279930,33.7628,-84.4220,6307261.0,0.288398,4.837758,0.018550,1.046730,0.288398,0.018550,NaN
1,2019-12-01,"Atlanta, GA",2388.0,32729.0,213.0,6732.0,4207.0,330.0,1368.872179,33.7628,-84.4220,6307261.0,0.378611,5.189099,0.033771,1.067341,0.667009,0.052321,NaN
2,2020-01-01,"Atlanta, GA",2912.0,2912.0,973.0,973.0,7119.0,1303.0,1368.798323,33.7628,-84.4220,6307261.0,0.461690,0.461690,0.154267,0.154267,1.128699,0.206587,NaN
3,2020-02-01,"Atlanta, GA",2742.0,5661.0,651.0,1562.0,9861.0,1954.0,1373.768759,33.7628,-84.4220,6307261.0,0.434737,0.897537,0.103214,0.247651,1.563436,0.309802,NaN
4,2020-03-01,"Atlanta, GA",2216.0,7883.0,127.0,1633.0,12077.0,2081.0,1382.102627,33.7628,-84.4220,6307261.0,0.351341,1.249829,0.020136,0.258908,1.914777,0.329937,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3095,2024-08-01,"Washington, DC",2137.0,15660.0,1257.0,7569.0,22460.0,11575.0,2375.472810,38.9047,-77.0163,6304975.0,0.338939,2.483753,0.199366,1.200481,3.562266,1.835852,0.050066
3096,2024-09-01,"Washington, DC",900.0,16477.0,222.0,7747.0,21885.0,11089.0,2383.224478,38.9047,-77.0163,6304975.0,0.142744,2.613333,0.035210,1.228712,3.471068,1.758770,0.048862
3097,2024-10-01,"Washington, DC",1801.0,19004.0,922.0,8651.0,21838.0,10897.0,2382.780218,38.9047,-77.0163,6304975.0,0.285647,3.014128,0.146234,1.372091,3.463614,1.728318,0.047143
3098,2024-11-01,"Washington, DC",1124.0,20058.0,318.0,8903.0,21457.0,10369.0,2372.798426,38.9047,-77.0163,6304975.0,0.178272,3.181297,0.050436,1.412060,3.403186,1.644574,0.044723


In [74]:
# sort by multi_rt_pc in the max date and filter to this date
homebuilding_zori[homebuilding_zori['date'] == homebuilding_zori['date'].max()].sort_values('rt_pc', ascending=False)

,date,name,total,total_ytd,multi_total,multi_total_ytd,rt,multi_rt,zori,lat,lng,pop_2023,total_pc,total_ytd_pc,multi_total_pc,multi_total_ytd_pc,rt_pc,multi_rt_pc,zori_yoy
123,2024-12-01,"Austin, TX",3073.0,32304.0,2020.0,16138.0,33530.0,16766.0,1675.390403,30.3005,-97.7522,2473275.0,1.242482,13.061224,0.816731,6.524952,13.556924,6.778866,-0.031384
2293,2024-12-01,"Raleigh, NC",893.0,18980.0,143.0,5727.0,19107.0,5786.0,1719.893690,35.8324,-78.6429,1509231.0,0.591692,12.575941,0.094750,3.794648,12.660090,3.833740,0.008075
1735,2024-12-01,"Nashville, TN",1260.0,19947.0,391.0,5626.0,19954.0,5750.0,1849.902241,36.1715,-86.7842,2102573.0,0.599266,9.486948,0.185963,2.675769,9.490277,2.734745,0.018306
433,2024-12-01,"Charlotte, NC",2102.0,25952.0,790.0,7205.0,25837.0,7122.0,1779.642907,35.2083,-80.8303,2805115.0,0.749345,9.251671,0.281628,2.568522,9.210674,2.538933,0.016711
1177,2024-12-01,"Jacksonville, FL",1064.0,15006.0,38.0,2144.0,15350.0,2169.0,1690.501961,30.3322,-81.6749,1713240.0,0.621046,8.758843,0.022180,1.251430,8.959632,1.266022,0.019776
2045,2024-12-01,"Phoenix, AZ",2322.0,45515.0,676.0,15735.0,44491.0,15002.0,1799.202542,33.5722,-112.0892,5070110.0,0.457978,8.977123,0.133330,3.103483,8.775155,2.958910,0.008447
743,2024-12-01,"Dallas, TX",6144.0,72151.0,2604.0,25721.0,70339.0,24897.0,1721.806844,32.7935,-96.7667,8100037.0,0.758515,8.907490,0.321480,3.175418,8.683787,3.073690,0.002405
1053,2024-12-01,"Houston, TX",4828.0,65871.0,980.0,13365.0,65155.0,12859.0,1680.787304,29.7860,-95.3885,7510253.0,0.642855,8.770810,0.130488,1.779567,8.675473,1.712193,0.027959
1921,2024-12-01,"Orlando, FL",2158.0,23644.0,865.0,8693.0,23522.0,8134.0,1988.895190,28.4773,-81.3370,2817933.0,0.765810,8.390547,0.306963,3.084885,8.347253,2.886513,0.015153
2975,2024-12-01,"Tampa, FL",949.0,20999.0,33.0,8048.0,21631.0,8015.0,2059.035759,27.9945,-82.4447,3342963.0,0.283880,6.281553,0.009871,2.407445,6.470607,2.397574,0.024493


In [79]:
# Create a 'state' column by extracting the state abbreviation from the 'name' column
homebuilding_zori['state'] = homebuilding_zori['name'].str.split(', ').str[-1]

homebuilding_zori

,date,name,total,total_ytd,multi_total,multi_total_ytd,rt,multi_rt,zori,lat,lng,pop_2023,total_pc,total_ytd_pc,multi_total_pc,multi_total_ytd_pc,rt_pc,multi_rt_pc,zori_yoy,state
0,2019-11-01,"Atlanta, GA",1819.0,30513.0,117.0,6602.0,1819.0,117.0,1371.279930,33.7628,-84.4220,6307261.0,0.288398,4.837758,0.018550,1.046730,0.288398,0.018550,NaN,GA
1,2019-12-01,"Atlanta, GA",2388.0,32729.0,213.0,6732.0,4207.0,330.0,1368.872179,33.7628,-84.4220,6307261.0,0.378611,5.189099,0.033771,1.067341,0.667009,0.052321,NaN,GA
2,2020-01-01,"Atlanta, GA",2912.0,2912.0,973.0,973.0,7119.0,1303.0,1368.798323,33.7628,-84.4220,6307261.0,0.461690,0.461690,0.154267,0.154267,1.128699,0.206587,NaN,GA
3,2020-02-01,"Atlanta, GA",2742.0,5661.0,651.0,1562.0,9861.0,1954.0,1373.768759,33.7628,-84.4220,6307261.0,0.434737,0.897537,0.103214,0.247651,1.563436,0.309802,NaN,GA
4,2020-03-01,"Atlanta, GA",2216.0,7883.0,127.0,1633.0,12077.0,2081.0,1382.102627,33.7628,-84.4220,6307261.0,0.351341,1.249829,0.020136,0.258908,1.914777,0.329937,NaN,GA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3095,2024-08-01,"Washington, DC",2137.0,15660.0,1257.0,7569.0,22460.0,11575.0,2375.472810,38.9047,-77.0163,6304975.0,0.338939,2.483753,0.199366,1.200481,3.562266,1.835852,0.050066,DC
3096,2024-09-01,"Washington, DC",900.0,16477.0,222.0,7747.0,21885.0,11089.0,2383.224478,38.9047,-77.0163,6304975.0,0.142744,2.613333,0.035210,1.228712,3.471068,1.758770,0.048862,DC
3097,2024-10-01,"Washington, DC",1801.0,19004.0,922.0,8651.0,21838.0,10897.0,2382.780218,38.9047,-77.0163,6304975.0,0.285647,3.014128,0.146234,1.372091,3.463614,1.728318,0.047143,DC
3098,2024-11-01,"Washington, DC",1124.0,20058.0,318.0,8903.0,21457.0,10369.0,2372.798426,38.9047,-77.0163,6304975.0,0.178272,3.181297,0.050436,1.412060,3.403186,1.644574,0.044723,DC


In [75]:
homebuilding_zori['name'].nunique()

50

In [77]:
regions = pd.read_csv('data/regions.csv')

regions

,state,region
0,WA,West
1,OR,West
2,CA,West
3,NV,West
4,ID,West
5,MT,West
6,WY,West
7,UT,West
8,CO,West
9,HI,West


In [80]:


# merge
homebuilding_zori = pd.merge(
    homebuilding_zori,
    regions,
    on='state',
    how='left'
)

homebuilding_zori

,date,name,total,total_ytd,multi_total,multi_total_ytd,rt,multi_rt,zori,lat,...,pop_2023,total_pc,total_ytd_pc,multi_total_pc,multi_total_ytd_pc,rt_pc,multi_rt_pc,zori_yoy,state,region
0,2019-11-01,"Atlanta, GA",1819.0,30513.0,117.0,6602.0,1819.0,117.0,1371.279930,33.7628,...,6307261.0,0.288398,4.837758,0.018550,1.046730,0.288398,0.018550,NaN,GA,Southeast
1,2019-12-01,"Atlanta, GA",2388.0,32729.0,213.0,6732.0,4207.0,330.0,1368.872179,33.7628,...,6307261.0,0.378611,5.189099,0.033771,1.067341,0.667009,0.052321,NaN,GA,Southeast
2,2020-01-01,"Atlanta, GA",2912.0,2912.0,973.0,973.0,7119.0,1303.0,1368.798323,33.7628,...,6307261.0,0.461690,0.461690,0.154267,0.154267,1.128699,0.206587,NaN,GA,Southeast
3,2020-02-01,"Atlanta, GA",2742.0,5661.0,651.0,1562.0,9861.0,1954.0,1373.768759,33.7628,...,6307261.0,0.434737,0.897537,0.103214,0.247651,1.563436,0.309802,NaN,GA,Southeast
4,2020-03-01,"Atlanta, GA",2216.0,7883.0,127.0,1633.0,12077.0,2081.0,1382.102627,33.7628,...,6307261.0,0.351341,1.249829,0.020136,0.258908,1.914777,0.329937,NaN,GA,Southeast
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3095,2024-08-01,"Washington, DC",2137.0,15660.0,1257.0,7569.0,22460.0,11575.0,2375.472810,38.9047,...,6304975.0,0.338939,2.483753,0.199366,1.200481,3.562266,1.835852,0.050066,DC,Northest
3096,2024-09-01,"Washington, DC",900.0,16477.0,222.0,7747.0,21885.0,11089.0,2383.224478,38.9047,...,6304975.0,0.142744,2.613333,0.035210,1.228712,3.471068,1.758770,0.048862,DC,Northest
3097,2024-10-01,"Washington, DC",1801.0,19004.0,922.0,8651.0,21838.0,10897.0,2382.780218,38.9047,...,6304975.0,0.285647,3.014128,0.146234,1.372091,3.463614,1.728318,0.047143,DC,Northest
3098,2024-11-01,"Washington, DC",1124.0,20058.0,318.0,8903.0,21457.0,10369.0,2372.798426,38.9047,...,6304975.0,0.178272,3.181297,0.050436,1.412060,3.403186,1.644574,0.044723,DC,Northest


In [81]:
# save the final dataset to a csv file
homebuilding_zori.to_csv('data/homebuilding_zori.csv', index=False)

In [44]:
# make a list of the names of every state and District of Columbia
us_states = ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'District of Columbia', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming']

In [51]:
nov24 = "https://www.census.gov/construction/bps/xls/statemonthly_202411.xls"

df = pd.read_excel(nov24, skiprows=5)

# rename columns
df.rename(
    columns={
        'Unnamed: 0': 'Name'
    },
    inplace=True
)

# drop null names
df = df.dropna(subset=['Name'])

# drop rows where 'Name includes 'Region', 'Division', or 'United States'
df = df[~df['Name'].str.contains('Region|Division|United States')]

# drop columns that have 'Unnamed' in the name
df = df.loc[:, ~df.columns.str.contains('^Unnamed|Year to Date')]

# add date column
df['date'] = '202411'

# make it datetime
df['date'] = pd.to_datetime(df['date'], format='%Y%m')

df

,Name,Total,1 Unit,2 Units,3 and 4 Units,5 Units or More,Num of Structures With 5 Units or More,date
7,Connecticut,564.0,146.0,2.0,9.0,407.0,15.0,2024-11-01
8,Maine,387.0,317.0,18.0,15.0,37.0,6.0,2024-11-01
9,Massachusetts,1006.0,355.0,24.0,6.0,621.0,36.0,2024-11-01
10,New Hampshire,336.0,224.0,10.0,19.0,83.0,7.0,2024-11-01
11,Rhode Island,90.0,63.0,24.0,3.0,0.0,0.0,2024-11-01
12,Vermont,132.0,112.0,2.0,6.0,12.0,2.0,2024-11-01
15,New Jersey,3179.0,1110.0,60.0,40.0,1969.0,84.0,2024-11-01
16,New York,3492.0,775.0,66.0,40.0,2611.0,86.0,2024-11-01
17,Pennsylvania,1800.0,1208.0,42.0,44.0,506.0,27.0,2024-11-01
21,Illinois,1819.0,779.0,58.0,87.0,895.0,34.0,2024-11-01
